In [1]:
#import codecs
#import glob
#import logging
#import multiprocessing
import os
#import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('..')
import time
import warnings

In [2]:
from data_preprocessing import data_cleaning
from data_preprocessing import *

### Configuration

In [ ]:
notes_path = '../../data/NOTEEVENTS.csv'
word_list_path = '../../data/label.npy'

### Getting all the notes and tokenization

In [ ]:
#use for filtering
label = np.load(word_list_path)
voca=label.item()['WORD_LIST']
voca_set = set(voca)

In [ ]:
notes_cleaned_pickle = pd.read_csv('debug_try.csv')

In [ ]:
notes_cleaned_pickle.head(1)

In [ ]:
notes_cleaned_pickle.shape

In [ ]:
notes = notes_cleaned_pickle['TEXT'].tolist()

In [ ]:
map_relation= data_cleaning.create_unique_identification_dictionary(notes,)
_, cleaned_notes = data_cleaning.identification_reform(notes,map_relation)
cleaned_notes = data_cleaning.clean_spaces(cleaned_notes)
tokens = data_cleaning.word_tokenize(cleaned_notes)

In [ ]:
map_relation

In [ ]:
len(map_relation)

In [ ]:
for i in tokens:
    for j in i:
        if j not in unique_token_list:
            unique_token_list.append(j)

In [ ]:
def filter_by_set(notes_list):
    '''
    notes_list: each note in a list
    voca: the dictionary we use
    return: filtered notes
    '''
    notes_set = set(notes_list)
    notes_vaild = notes_set & voca_set
    new_li=[i for i in notes_list if i in notes_vaild]
    return new_li

def filter_word(notes_list, voca):
    '''
    notes_list: each note in a list
    voca: the dictionary we use
    return: filtered notes
    '''
    new_li=[i for i in notes_list if i in voca]
    return new_li

In [ ]:
embed_dim = 300
min_word_count = 3
#num_workers = multiprocessing.cpu_count()
context_size = 7
downsampling = 1e-3
seed = 134

In [ ]:
num_workers
#a sentence should be a list of words

In [ ]:
medical2vec = w2v.Word2Vec(tokens,sg = 1, seed = seed,  size = embed_dim, min_count = min_word_count, \
                           window = context_size, sample = downsampling)

In [ ]:
medical2vec.save('model_word2vec.bin')

In [ ]:
print(medical2vec)
medical2vec.build_vocab(tokens)#must build vocabulary before train model
medical2vec.train(tokens)
medical2vec.save(os.path.join("trained", "medical2vec.w2v"),ignore=[])

### Readin the Model

In [ ]:
new_voca = np.load('../../data/voc_100.npy')

In [ ]:
model = w2v.Word2Vec.load('model_word2vec_mode1.bin')

In [ ]:
list(new_voca)

In [ ]:
word_vectors = model.wv

In [ ]:
print(model.wv.index2word[0], model.wv.index2word[1], model.wv.index2word[2])

In [ ]:
model.wv['welldifferentiated']

In [ ]:
model2 = w2v.Word2Vec.load('model_word2vec_mode2.bin')

In [ ]:
print(model2.wv.index2word[0], model2.wv.index2word[1], model2.wv.index2word[2])

In [ ]:
len(model2.wv.index2word),len(model.wv.index2word),len(voca_set)

In [ ]:
model2['apacingpatient']

In [ ]:
len(new_voca)

In [ ]:
112514 - 77386

In [ ]:
num

In [ ]:
77386 + 59769

In [ ]:
num = 0
for i in new_voca:
    try:
        a = model[i]
    except KeyError:
        num += 1        

In [ ]:
num = 0
for i in voca_set:
    try:
        a = model2[i]
    except KeyError:
        num += 1        

In [ ]:
num

### NEW_read cleaned_csv

In [ ]:
df = pd.read_csv('/Users/leilei/Documents/DS1011Project/data/cleaned_notes.csv')

In [ ]:
df.head()

In [ ]:
import string 
translator = str.maketrans('', '', string.punctuation.replace(".",""))

### New_w2v

In [3]:
model = w2v.Word2Vec.load('./model_word2vec_mode1_cleaned_notes.bin')

word2idx = np.load('word_to_ix.npy')

new_voca = np.load('../../data/voc_100.npy')

#index from 1
word2idx = word2idx.reshape(1,-1)[0][0]

In [11]:
len(word2idx)

112514

In [18]:
len(new_voca)

112514

In [6]:
num = 0
for i in new_voca:
    try:
        a = model[i]
    except KeyError:
        print(i)
        num += 1      

.


In [7]:
num

1

In [20]:
embedding_mtrx = np.zeros((112514, 300))
random = np.random.random(300)
for i in word2idx.keys():#i is a word
    try:
        if i != '.':
            embedding_mtrx[word2idx[i]-1] = model[i]
        else:
            embedding_mtrx[word2idx[i]-1] = random
    except IndexError:
        print(i)

#embedding_mtrx[112514] = random
#embedding_mtrx[112515] 

In [22]:
import pickle
pickle.dump(embedding_mtrx, open( "../../data/embedding_mtrx.p", "wb" ),protocol=2 )